## Import the required libraries and dependencies.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load the dataset for use.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/train',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory('/content/drive/MyDrive/validation',
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')

In [ ]:

# Print the class labels in the train generator
print("Class labels in the train generator:")
print(train_generator.class_indices)


In [ ]:

# Print the class labels in the test generator
print("Class labels in the test generator:")
print(valid_generator.class_indices)


## Build an AlexNet.

In [ ]:
model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
model.add(layers.Conv2D(256, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
model.add(layers.Conv2D(384, (3, 3), activation='relu'))
model.add(layers.Conv2D(384, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))

# Flatten layer
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(11, activation='softmax'))  # 11 classes
target_size=(224, 224)
print(train_generator.image_shape)
model.summary()


## Compile the model.

In [ ]:
#Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
history=model.fit(train_generator,
          epochs=10,   #number of epochs
          validation_data=valid_generator)

## Calculate model accuracy.

In [ ]:
accuracy = history.history['val_accuracy'][-1]

print(f'Test Accuracy: {accuracy*100:.4f}%')

## Save the model to disk.

In [ ]:
print("[INFO] Saving model...")
model.save("epic model.h5")
print("[INFO] Saved model to disk!")

## Plot the accuracy and loss curves.


In [ ]:
# Plot the training and validation loss and accuracy curves
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot the smoothed training loss
axs[0].plot(train_loss, label='Training Loss')
axs[0].plot(val_loss, label='Validation Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()

# Plot the smoothed training accuracy
axs[1].plot(train_acc, label='Training Accuracy')
axs[1].plot(val_acc, label='Validation Accuracy')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].legend()

plt.show()

## Test the model.

In [ ]:
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
# Load the saved model
model = tf.keras.models.load_model('model.h5')

# Load and preprocess a test image
test_image_path = 'image.jpeg'
# Load the image
img = Image.open(test_image_path)
# Display the image using matplotlib
plt.imshow(img)
plt.axis('off')  # Turn off axis labels
plt.show()
test_image = image.load_img(test_image_path, target_size=(224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image /= 255.0  # Normalize the pixel values


class_name_dict = {
    0: 'Bacterial spot',
    1: 'Early blight',
    2: 'Fusarium wilt',
    3: 'Late blight',
    4: 'Leaf mold',
    5: 'Septoria leaf spot',
    6: 'Spider mites',
    7: 'Target spot',
    8: 'Yellow leaf curl virus',
    9: 'Mosaic virus',
    10: 'Healthy'
}


# Make predictions
predictions = model.predict(test_image)
predicted_class = np.argmax(predictions)
predicted_class_name = class_name_dict[predicted_class]

# Display the predicted class name
print(f"Predicted Class: {predicted_class_name}")
